In [1]:
import pandas as pd
import torch
import pytorch_lightning as pl
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

In [2]:
candidates = pd.read_csv("recsys22/data/candidate_items.csv")
train_purchases = pd.read_csv("recsys22/data/train_purchases.csv")
train_sessions = pd.read_csv("recsys22/data/train_sessions.csv")

all_interactions = pd.concat([train_sessions, train_purchases], ignore_index=True)

In [3]:
n_session, n_item = all_interactions.session_id.max(), all_interactions.item_id.max(), 

In [4]:
from torchrecsys.datasets import InteractionsDataset
from torchrecsys.models import BaseModel
from torchrecsys.task import Ranking
import torchrecsys as trs

In [5]:
ds =  InteractionsDataset(all_interactions[["session_id","item_id"]].values)
train_dataloader = torch.utils.data.DataLoader(ds, batch_size=64, shuffle=True)

In [6]:
for a in train_dataloader:
    break

In [7]:
class retrievalModel(trs.BaseModel):
    def __init__(self):
        super().__init__()
        self.session_model = torch.nn.Embedding(n_session, 64)
        self.item_model = torch.nn.Embedding(n_item, 64)
        self.task = Ranking()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)

    def forward(self, x):
        # We pick out the user features and pass them into the user model.
        session_embeddings = self.session_model(x[0])
        # And pick out the movie features and pass them into the movie model,
        # getting embeddings back.
        item_embeddings = self.item_model(x[1])

        # The task computes the loss and the metrics.
        # return session_embeddings, item_embeddings
        return session_embeddings, item_embeddings
    
    
model = retrievalModel()
yhat = model(a)

In [8]:
# training
trainer = pl.Trainer(precision=32)
trainer.fit(model, train_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name          | Type      | Params
--------------------------------------------
0 | session_model | Embedding | 284 M 
1 | item_model    | Embedding | 1.8 M 
--------------------------------------------
285 M     Trainable params
0         Non-trainable params
285 M     Total params
1,143.845 Total estimated model params size (MB)
/Users/jafen/miniconda3/envs/hm37/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: 0it [00:00, ?it/s]

/Users/jafen/projects/github/torchrecsys/torchrecsys/task/ranking.py:15: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  labels = torch.range(0, num_queries-1, dtype=int)
/Users/jafen/miniconda3/envs/hm37/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
